In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import os
import time
import io
import json
import pyathena as cz
from pyathena.pandas_cursor import PandasCursor
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_similarity_score as jss # evaluation metric
from sklearn.metrics import precision_score # evaluation metric
from sklearn.metrics import classification_report # evaluation metric
from sklearn.metrics import confusion_matrix # evaluation metric
from sklearn.metrics import log_loss # evaluation metric
import pickle
import time
import awswrangler as wgl

ModuleNotFoundError: No module named 'awswrangler'

In [ ]:
# s3 = boto3.resource('s3')
athena_conection = "athena.us-east-1.amazonaws.com"
region_name = ('us-east-1')

In [20]:
client   = boto3.client('athena', region_name='us-east-1', aws_access_key_id=os.environ['ACCESS_KEY'],
                            aws_secret_access_key=os.environ['ACCESS_SECRET_KEY'])

s3_resource   = boto3.resource('s3', region_name='us-east-1', aws_access_key_id=os.environ['ACCESS_KEY'],
                            aws_secret_access_key=os.environ['ACCESS_SECRET_KEY'])

In [47]:
def athena_query_to_dataframe(db, s3Bucket, query):
    
    import boto3
    import pandas as pd
    
    listOfStatus = ['SUCCEEDED', 'FAILED', 'CANCELLED']
    listOfInitialStatus = ['RUNNING', 'QUEUED']
    
    print('Starting Query Execution:')
    
    tempS3Path = 's3://{}'.format(s3Bucket)
    
    response = client.start_query_execution(
        QueryString = query,
        QueryExecutionContext = {
            'Database': db
        },
        ResultConfiguration = {
            'OutputLocation': tempS3Path,
        }
    )

    queryExecutionId = response['QueryExecutionId']

    status = client.get_query_execution(QueryExecutionId = queryExecutionId)['QueryExecution']['Status']['State']

    while status in listOfInitialStatus:
        status = client.get_query_execution(QueryExecutionId = queryExecutionId)['QueryExecution']['Status']['State']
        if status in listOfStatus:
            if status == 'SUCCEEDED':
                print('Query Succeeded!')
                paginator = client.get_paginator('get_query_results')
                query_results = paginator.paginate(
                    QueryExecutionId = queryExecutionId,
                    PaginationConfig = {'PageSize': 1000}
                )
            elif status == 'FAILED':
                print('Query Failed!')
            elif status == 'CANCELLED':
                print('Query Cancelled!')
            break
    
    results = []
    rows = []
    
    print('Processing Response')
    
    for page in query_results:
        for row in page['ResultSet']['Rows']:
            rows.append(row['Data'])

    columns = rows[0]
    rows = rows[1:]

    columns_list = []
    for column in columns:
        columns_list.append(column['VarCharValue'])
        
    print('Creating Dataframe')

    dataframe = pd.DataFrame(columns = columns_list)

    for row in rows:
        df_row = []
        try:
            for data in row:
                df_row.append(data['VarCharValue'])
            dataframe.loc[len(dataframe)] = df_row
        except:
            pass
        
    return dataframe  

In [48]:
df = athena_query_to_dataframe("ml_platform_events", "personal-us-east-1/query-result/" ,"select * from ml_platform_events.cleaned_punkapi")

Starting Query Execution:
Query Succeeded!
Processing Response
Creating Dataframe


In [49]:
df.head()

,id,name,abv,ibu,target_fg,target_og,ebc,srm,ph
0,159,Black Tokyo Horizon (w/Nøgne Ø & Mikkeller),17.2,75.0,1030,1156,128.0,64.0,4.4
1,165,Barley Wine - Russian Doll,10.0,80.0,1020,1097,65.0,32.5,4.4
2,203,Bitch Please (w/ 3 Floyds),11.5,50.0,1016,1104,38.0,19.0,4.4
3,25,Bad Pixie,4.7,45.0,1010,1047,8.0,4.0,4.4
4,275,Sidewalk Shark,5.2,22.0,1008,1055,7.0,4.0,3.8


In [53]:
def trainning_classfication():
    model = LogisticRegression()
        
    X = df[['abv', 'target_fg', 'target_og', 'ebc', 'srm', 'ph']]
    y = df['ibu']
    
    output = []
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    
    model.fit(X_train, y_train)
    
#     lr_predict = logisticRegr.predict(X_test)
#     lr_proba = logisticRegr.predict_proba(X_test)
    
#     print('Jaccard Similarity Score of our model is {}'.format(jss(y_test, lr_predict).round(2)))
    
    ## save on local
    filename = 'model_punkbeer.pkl'
    pickle.dump(model, open(filename, 'wb'))

    metadata = {
        'model': json.dumps(model.__class__.__name__),
        'path': filename
    }
    
    m = json.dumps(metadata)
    
    fh = open("metadata.json", 'w')
    json.dump(m, fh)
    
    return metadata

In [54]:
p = trainning_classfication()
print(p)

{'model': '"LogisticRegression"', 'path': 'model_punkbeer.pkl'}


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
